In [8]:
import csv
import sys
import numpy as np
import math
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from math import sqrt

d = pd.read_csv("raw.csv",index_col=False)


In [9]:
def pega(sentence):
    sent_str = ""
    for i in sentence:
        sent_str += str(i)
    return sent_str
def cat(outfilename, line):
    with open(outfilename, 'w') as outfile:
            outfile.write(line)
            outfile.close()
def cat2(outfilename, line):
    with open(outfilename,'a') as outfile:
            outfile.write(line)
            outfile.close()

In [12]:
prueba_numero =4

f = 100
p = 50
prefix ='C:/Users/Celia/Desktop/4º/TFG/codigo/python'
path = pega([prefix,'/data20_10_scaled_inc',prueba_numero,'.txt'])

cat(path,pega(["p: ",p," f: ",f,"\n"]))
errores =pega([prefix,'/rmse',prueba_numero,'.txt'])
cat(errores,pega(["Rmse mejores",prueba_numero,"\n"]))
#archivo registro de pruebas
n=pega([prefix,'/registro_pruebas.txt'])
cat2(n,pega(['nº',prueba_numero ,' columna_predecir: BDIY_Index_Open ',"p: ",p," f: ",f," columnas: ",len(d.columns)," n: ",3]))


max_col = 3


In [ ]:

for col in list(d.columns.values):
    print(col)
    if(col=='BDIY_Index_Open'):# probar solo con euro
        
        present = d[(p):(len(d)-f)][col].reset_index()
        future = d[(f+p):len(d)][col].reset_index()
        # la etiqueta es el % de incremento en f días (saltando los p+1 primeros)

        label1 = future.subtract(present, fill_value=0).reset_index()
        label2 = (label1[col]/present[col]).reset_index()
        label=label2.drop(label2.columns[[0]], axis='columns')


        array = [0] * len(label)
        zeros = pd.DataFrame(data=array)

        a = label.values

        rmse =sqrt(mean_squared_error(array, a))
        cat2(path,pega([col," ", rmse,"\n"]))
        # hasta aqui columna label y error del naive
        
        rmse_up_to_now = 1
        columns= []
        column_names = []

        n = 0
        puntoFijo = False
        cambios = 0
        # miemtras que no se lleven el máximo de columnas y siga mejorando
        while n < max_col and (not puntoFijo):
            print(pega(["n=", n]))
            ndf = columns
            ini = 0
            fin = len(d.columns)
            
            #recorre todas las columnas
            for i in np.arange(ini, fin):
                print(i)
                #nombre de la columna
                nombres = pega([list(d.columns.values)[i],1])
                #si esa columna no ha sido cogida ya antes
               
                if not (nombres in column_names):
                    pred = i
                    s = np.arange(1, len(d)+1)# filas de la tabla
                    vs = np.arange(0, p) # pasados a utilizar es ahasta p-1 porque no incluyepe la funcion arange
                    name = list(d.columns.values)[pred]
                    names = []
                    #para la columna  i con la que quiero hacer la prediccion  en names estan los posibles pasados de esa columna a usar
                    for v in vs:
                        names.append(pega([name, (p-v)]))
                    df=pd.DataFrame(columns=names)
                    # desde el dia p+1(cosas de indices) hasta el final menos futuro --> tabla
                    for i in np.arange(p,len(d)-f):
                        #para cada filla nrwo representa la fila actual
                        nrow=[]
                        #rellenar las columnas de (EuroPasado10, EuroPasado9, EuroPadaso8....)
                        for v in vs:
                            #calcular los incrementos que ha habido  en un marco temporal de 10 dias atras
                            #d[ el dia a predeccir - (p-v= rango de 10 a 1) dias atras]
                            nrow.append(((d.iloc[i-(p-v)][pred])-(d.iloc[i][pred]))/ (d.iloc[i][pred]))
                        df.append(nrow)
                        df = df.append(pd.Series(nrow, index=df.columns ), ignore_index=True)
                     
                    
                    
                    # se añade el mejor conjunto de columnas de la iteracion anterior de n si no es vacio
                    if(len(columns)!=0):
                        df=pd.concat([columns, df], axis=1, sort=False)
                    
                    # crear la matriz de datos con label
                    datos = pd.concat([label, df], axis=1, sort=False)
                    
                   
                    
                    
                    #-----------------EVALUACION---------------------------------------
                   
                        
                            
                   
                    
                    trainsize=1500 # tamaño de inicion del conjunto de entrenamiento
                    s =np.arange(trainsize,len(datos)-f)
                   
                    predichos = []
                    reales=[]
                    for j in s:
                        train = datos[1:j][:]
                        lr = linear_model.LinearRegression()
                        lr.fit(train.iloc[:,1:], train.iloc[:,0:1])
                        
                        test=datos.iloc[[j+f],1:]
                        Y_pred = lr.predict(test)
                        real =datos.iloc[[j+f],0:1]
                        
                        valor_p =Y_pred[0][0]
                        predichos.append(valor_p)
                        valor_r=real.iloc[0,0]
                        reales.append(valor_r)
                    
                    
                    rmse =sqrt(mean_squared_error(reales, predichos))
                    cambios +=1
                    
                    #si mejora
                    if(not(np.isnan(rmse)) and rmse<rmse_up_to_now):
                        rmse_up_to_now=rmse
                        ndf= df
                        cat2(errores,pega([col, " con ", name," iteracion", n," ", rmse_up_to_now,"\n"]))
                
                        
             
            print(pega(["mejores columnas iteracion: ",n]))
            print(ndf.columns.values)
                  
            # aqui termina la busqueda de la combinacion de pasasdos para las columnas i para predecir la columna col         
            if(cambios>0 or len(columns==0) or not (columns.columns.values ==ndf.columns.values) ) :
               
                columns = ndf
              
                columns_names = columns.columns.values
                
                print("cambio columnas")
                print(column_names)
                n+=1
            else:
                print("vuelta sin cabios")
                puntoFijo = True
                    
        print("Acabado")
        print(rmse_up_to_now)
        print(ndf.columns.values)
    
                    
               
                    
               
                    
                   
                    
                    
                    
                    
          

   

    
    
     


    
    
   
    
    
    


EUR_Curncy_Open
EUR_Curncy_Closing
EUR_Curncy_High
EUR_Curncy_Low
EUR_Curncy_Volatility
JPY_Curncy_Open
JPY_Curncy_Closing
JPY_Curncy_High
JPY_Curncy_Low
JPY_Curncy_Volatility
GBP_Curncy_Open
GBP_Curncy_Closing
GBP_Curncy_High
GBP_Curncy_Low
GBP_Curncy_Volatility
CHF_Curncy_Open
CHF_Curncy_Closing
CHF_Curncy_High
CHF_Curncy_Low
CHF_Curncy_Volatility
CAD_Curncy_Open
CAD_Curncy_Closing
CAD_Curncy_High
CAD_Curncy_Low
CAD_Curncy_Volatility
NOK_Curncy_Open
NOK_Curncy_Closing
NOK_Curncy_High
NOK_Curncy_Low
NOK_Curncy_Volatility
INDU_Index_Open
INDU_Index_Closing
INDU_Index_High
INDU_Index_Low
INDU_Index_Volatility
SPX_Index_Open
SPX_Index_Closing
SPX_Index_High
SPX_Index_Low
SPX_Index_Volatility
CCMP_Index_Open
CCMP_Index_Closing
CCMP_Index_High
CCMP_Index_Low
CCMP_Index_Volatility
SPTSX_Index_Open
SPTSX_Index_Closing
SPTSX_Index_High
SPTSX_Index_Low
SPTSX_Index_Volatility
MEXBOL_Index_Open
MEXBOL_Index_Closing
MEXBOL_Index_High
MEXBOL_Index_Low
MEXBOL_Index_Volatility
UKX_Index_Open
UKX_Ind

C:\Users\Celia\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexes\api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197


In [ ]:
print(d.columns.values)

In [ ]:
df.iloc[4][5]